In [1]:
import sys
import os
import pandas as pd
import numpy as np
import datetime as dt
import json
import geojson
import requests
import sqlite3 as db
sys.path.append('../')

pd.options.display.max_rows=99

In [4]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'
##cur = cnx.cursor()

In [5]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

NASA Data Products:
- LAI: LAI is a measure for the total area of leaves per unit ground area and directly related to the amount of light that can be intercepted by plants. It is defined as the one-sided green leaf area per unit ground surface area (LAI = leaf area / ground area, m2 / m2) in broadleaf canopies. There are three methods used to measure LAI for conifers; this project uses projected (or one-sided, in accordance the definition for broadleaf canopies) needle area per unit ground area.
    - In general, a higher LAI value indicates more leaf coverage
- fPAR: Fraction of absorbed photosynthetically active radiation (fPAR) is the fraction of incoming solar radiation in the spectrum of 400–700 nm that is absorbed by vegetation canopy. Data is provided as a percentage.
- Land Cover Type 3: Annual Leaf Area Index (LAI) classification
- FAO-Land Cover Classification System 1 (LCCS1) land cover layer

In [6]:
diff = 0.005    ##equivalent to 0.5 miles when applied
hotspotsGeo = pd.DataFrame()
hotspotsGeo = pd.read_sql('SELECT locId,lat,lng FROM Hotspots', con=connectDB())
##TODO #34 remove duplicate hotspots before building table of polygons for each hotspot --DONE
##hotspotsGeo.set_index('index',inplace=True)
hotspotsGeo.sort_values(by=['locId'],ascending=True,inplace=True)
hotspotsGeo.drop_duplicates(subset=['locId'],keep='first',inplace=True)
hotspotsGeo.reset_index()
##TODO #33 write a function to increment the lat & lng values as needed then output the results to a single column in the format lat,lng --DONE

##Define functions to produce a corner of a polygon for each hotspot (NE,SE,SW,NW)
def NW(x,y):
    return x-diff,y+diff
def NE(x,y):
    return x+diff,y+diff
def SE(x,y):
    return x+diff,y-diff
def SW(x,y):
    return x-diff,y-diff

##apply the functions as new columns
##NOTE that appEEARS only accepts coordinates as (longitude,latitude) which is contrary to geoJSON documentation
hotspotsGeo['NW'] = hotspotsGeo.apply(lambda i: NW(i.lng,i.lat), axis = 1)##.astype(str)
hotspotsGeo['NE'] = hotspotsGeo.apply(lambda i: NE(i.lng,i.lat), axis = 1)##.astype(str)
hotspotsGeo['SE'] = hotspotsGeo.apply(lambda i: SE(i.lng,i.lat), axis = 1)##.astype(str)
hotspotsGeo['SW'] = hotspotsGeo.apply(lambda i: SW(i.lng,i.lat), axis = 1)##.astype(str)
##TODO #35 (script to support) write the results to the database as a new table with a 'latestUpdate' column --DONE
today = dt.datetime.today()
hotspotsGeo['latestUpdate'] = today.date()
hotspotsGeo['latestUpdate'] = hotspotsGeo['latestUpdate'].astype('datetime64[ns]')
##hotspotsGeo.to_sql(name='hotspotsGeo',con=cnx,if_exists='append')

##TODO #37 compile the polygon generated for each hotspot into a coordinate pack formatted for JSON insert --DONE
##no longer neccessary, geoJSON handles packaging
##for locId in hotspotsGeo.itertuples():
    ##hotspotsGeo['polygon'] = '[[{},{},{},{},{}]]'.format(locId.NW,locId.NE,locId.SE,locId.SW,locId.NW).replace('(','[').replace(')',']')

In [7]:
##TODO #40 remove the getpass prompt and replace with a credential manager call
earthdata_baseUrl = 'https://lpdaacsvc.cr.usgs.gov/appeears/api/'
appEEARS_username = 'lwylie'
appEEARS_password = 'BdiUPBhUa7ma5ds'
import getpass
NASA_username = getpass.getpass(prompt = 'Enter NASA Earthdata Login Username: ')
NASA_password = getpass.getpass(prompt = 'Enter NASA Earthdata Login Password: ')

In [8]:
##obtain an Earthdata token
##TODO #36 the earthdata API is often under maintenance, write a script to abort this process if a new token cannot be obtain
earthdata_loginURL = 'https://lpdaacsvc.cr.usgs.gov/appeears/api/login'
earthdata_loginRequest = requests.post(earthdata_loginURL,auth=(NASA_username,NASA_password))
earthdata_loginResponse = earthdata_loginRequest.json()
print(earthdata_loginResponse)
##Transcribe token, builder header
earthdata_token = earthdata_loginResponse['token']
earthdata_head = {'Authorization': 'Bearer {}'.format(earthdata_token)}

{'token_type': 'Bearer', 'token': '_XZ-yiaeVWRoKjWtV8d2PpIeqe54GcjjvxGOebN0nnlC7rP4X88SOiCAej_XWHcrAxRO38lCgEZNkOyidkwnZQ', 'expiration': '2022-02-19T18:00:30Z'}


In [ ]:
##TODO #43 write a cooked table of locId's with stats to check against list of hotspots. Drive fetch off difference

Architecture:

1. Make all requests in a loop, producing a list of {'locId': 'earthdata_taskID'}
2. For each locId, pull each .csv into a dataframe then load that dataframe into a database table bearing the name that corresponds with the layer and product. Append the locId.
    EXAMPLE: the contents of the .csv file for 'MCD12Q1-006-LC-Prop1-Statistics.csv' go into the table 'MCD12Q1-006-LC-Prop1-Statistics' in the database

In [9]:
##small frame of hotspots for testing
hotspotsGeo_short = hotspotsGeo.head(2)

In [50]:
def geoPack(NW,NE,SE,SW):
    geoPack_wrap = {"type": "FeatureCollection", "features":
        [{
        "type":"Feature",
            "geometry":
                {"type": "Polygon",
                "coordinates":
                    [[NW,NE,SE,SW,NW]]
                },
            "properties": {}}]
        }
    return geojson.GeoJSON(geoPack_wrap)

In [16]:
def task(taskName: str,endDate: str,startDate: str,recurring: bool,yearRange: list, geoPack_wrap = geojson.GeoJSON):
    
    edTask = {
        'task_type': 'area',
        'task_name': taskName,
        'params': {
            'dates': 
                [{"endDate": endDate, 
                "recurring": recurring, 
                "startDate": startDate, 
                "yearRange": yearRange}],
            'layers': 
                [{"layer": "LC_Prop1", "product": "MCD12Q1.006"}, 
                {"layer": "LC_Type3", "product": "MCD12Q1.006"}],
            'output': {
                'format': {
                    'type': 'netcdf4'}, 
                    'projection': 'geographic'},
        'geo':geoPack_wrap}}
    return edTask

In [56]:
def post_earthdataTask():
    earthdataTaskList = []
    try:
        for locId in hotspotsGeo_short.itertuples():
            coords = geoPack(NW = locId.NW,NE = locId.NE,SE = locId.SE,SW = locId.SW)
            earthdataTask = task(taskName = locId.locId,endDate = "12-31",startDate="01-01",recurring=True,yearRange=[2017,2019],geoPack_wrap=coords)
            taskReq = requests.post(f'{earthdata_baseUrl}task',json=earthdataTask,headers=earthdata_head).json()
            earthdataTaskList.append(taskReq)
    except Exception as ee:
        raise UserWarning(ee)
    return earthdataTaskList

In [57]:
help = post_earthdataTask()

In [58]:
help

[{'task_id': '48f1b070-2dd8-47f1-b7dd-3003f05ecc40', 'status': 'pending'},
 {'task_id': 'd4d0be89-3b6b-454b-a9bb-f2a5bfc6c98e', 'status': 'pending'}]

In [ ]:
##TODO #50 write a function that splits out bad responses from appEEARS and logs them to the server

In [ ]:
##TODO #44 resolve error: {'message': "RecursionError('maximum recursion depth exceeded in __instancecheck__',)"} --DONE
##TODO #45 once all hotspots/locId's have Prop1 and Type3 MCD12Q1 data, convert this block to check for LAI and fPAR

    
##TODO #42 if the request returns anything other than a task_id, stop the script and write the returned issue to the error log --DONE


In [105]:
taskid = pd.DataFrame(help)

In [ ]:
##submit the task request
##TODO #42 if the request returns anything other than a task_id, stop the script and write the returned issue to the error log
params = {'limit': 15, 'pretty': True}
earthdata_tasks = requests.get('{}task'.format(earthdata_baseUrl),params=params,headers=earthdata_head).json()

In [ ]:
##TODO #51 write a function that takes the results of get_earthdataTask(), fetches the .csv, returns a DataFrame and then pushes that DataFrame to the database.

In [152]:
def get_earthdataTask():
    postedTasks = pd.DataFrame(help)
    taskInfo = []
    cnx = connectDB()
    ##postedTasks = pd.DataFrame(post_earthdata_Task())
    import time
    try:
        for task_id in postedTasks.itertuples():
            taskStatus = requests.get(f'{earthdata_baseUrl}task/{task_id.task_id}',headers=earthdata_head).json()
            starttime = time.time()
            while taskStatus['status'] !='done':
                print(requests.get(f'{earthdata_baseUrl}task/{task_id.task_id}',headers=earthdata_head).json()['status'])
                time.sleep(20.0 - ((time.time() - starttime) % 20.0))
            taskInfo.append(taskStatus)
        taskBatch = pd.DataFrame(data=taskInfo,columns=['task_id','task_name','status','completed'])

        for task_id in taskBatch.itertuples():
            earthdata_bundle = requests.get(f'{earthdata_baseUrl}bundle/{task_id.task_id}').json()
            ##filter down to .csv files
            fileList = []
            for x in earthdata_bundle['files']:
                if x['file_type'] in 'csv':
                    time.sleep(0.3)
                    fileID = x['file_id']
                    fileName = x['file_name']
                    ##To read results to a dataframe, pass in earthdata_baseUrl + 'bundle/ + taskID/ + file_id/ + file_name
                    getIt = pd.DataFrame(pd.read_csv(f'{earthdata_baseUrl}bundle/{task_id.task_id}/{fileID}/{fileName}'))
                    ##TODO #52 almost there, make this script below function:
                    ##getIt = pd.read_csv(f'{earthdata_baseUrl}bundle/{task_id.task_id}/{fileID}/{fileName}')
                    ##getIt.to_sql(name='MODIS_cooking',con=cnx,if_exists='append')
                    
                else: continue
                allgets = pd.concat([getIt])     
    except Exception as ex:
        raise UserWarning(ex)
    return allgets

In [153]:
tt = get_earthdataTask()
tt

,File,aid,Date,(7) Evergreen Needleleaf Forests,(10) Urban and Built-up Lands
0,LC_Type3_2017001_aid0001,aid0001,2017-01-01,2,7
1,LC_Type3_2018001_aid0001,aid0001,2018-01-01,2,7
2,LC_Type3_2019001_aid0001,aid0001,2019-01-01,2,7


In [135]:
##TODO #38 get the results of the appEEARS task as a bundle
earthdata_taskID = '48f1b070-2dd8-47f1-b7dd-3003f05ecc40'
earthdata_bundle = requests.get('{}bundle/{}'.format(earthdata_baseUrl,earthdata_taskID),).json()
earthdata_bundle

{'files': [{'sha256': '58fb0e19dfbd85ad0f636fa525b44ffa272bc212371dd74c42626f1823681fb7',
   'file_id': 'ca0e52f7-3f80-4777-8b7c-507d09f245dd',
   'file_name': 'MCD12Q1.006_500m_aid0001.nc',
   'file_size': 35199,
   'file_type': 'nc'},
  {'sha256': 'd29d9a15144007340d7b2df860d3b1d9226f1eb645b2df3527b720722ceec1f4',
   'file_id': 'b1e45e8d-1548-4993-bb5f-f9e46ed7fc29',
   'file_name': 'MCD12Q1-006-QC-lookup.csv',
   'file_size': 99,
   'file_type': 'csv'},
  {'sha256': 'ed3ab9b81e7c0b98695bc53839cd4ec2cef4b78ab23cf287232d82d53b6e72a5',
   'file_id': 'e6ab2e68-916b-4866-a260-da30730af562',
   'file_name': 'MCD12Q1-006-QC-Statistics-QA.csv',
   'file_size': 136,
   'file_type': 'csv'},
  {'sha256': '82fa1a7931a7f04ad1254e1427f6ee271bca68b66fa362267abd609c7194926d',
   'file_id': 'f9c5ec31-096e-48be-b0e5-f9a2bbe963b7',
   'file_name': 'MCD12Q1-006-LC-Prop1-Statistics.csv',
   'file_size': 235,
   'file_type': 'csv'},
  {'sha256': '79b145aed0fc5a10f4d65f69af9959b88900189c34605e0e2312ddb9de

In [144]:
earthdata_bundle['files']['file_name']

TypeError: list indices must be integers or slices, not str

In [141]:
##filter for .csv's
somelist = []
earthdata_files = {}
for f in earthdata_bundle['files']:
    if f['file_type'] in 'csv':
        somelist.append(f)
    else: continue


somelist

[{'sha256': 'd29d9a15144007340d7b2df860d3b1d9226f1eb645b2df3527b720722ceec1f4',
  'file_id': 'b1e45e8d-1548-4993-bb5f-f9e46ed7fc29',
  'file_name': 'MCD12Q1-006-QC-lookup.csv',
  'file_size': 99,
  'file_type': 'csv'},
 {'sha256': 'ed3ab9b81e7c0b98695bc53839cd4ec2cef4b78ab23cf287232d82d53b6e72a5',
  'file_id': 'e6ab2e68-916b-4866-a260-da30730af562',
  'file_name': 'MCD12Q1-006-QC-Statistics-QA.csv',
  'file_size': 136,
  'file_type': 'csv'},
 {'sha256': '82fa1a7931a7f04ad1254e1427f6ee271bca68b66fa362267abd609c7194926d',
  'file_id': 'f9c5ec31-096e-48be-b0e5-f9a2bbe963b7',
  'file_name': 'MCD12Q1-006-LC-Prop1-Statistics.csv',
  'file_size': 235,
  'file_type': 'csv'},
 {'sha256': '79b145aed0fc5a10f4d65f69af9959b88900189c34605e0e2312ddb9de96c31f',
  'file_id': '6309a3d1-ac26-498d-bee2-03d03f2bbe09',
  'file_name': 'MCD12Q1-006-LC-Type3-Statistics.csv',
  'file_size': 221,
  'file_type': 'csv'}]

In [ ]:
##??
for file_id in earthdata_files:
    allresults = pd.DataFrame()

In [ ]:
##IT WORKS
##To read results to a dataframe, pass in earthdata_baseUrl + 'bundle/ + taskID/ + file_id/ + file_name
##example: https://lpdaacsvc.cr.usgs.gov/appeears/api/bundle/098dbb7a-0dfc-4f19-8410-a1db4f91170c/5a51d0f8-362a-4ea3-9e8e-ce1bd783aa62/MCD12Q1-006-LC-Prop1-Statistics.csv
qrf = pd.DataFrame()
qrf = pd.read_csv('https://lpdaacsvc.cr.usgs.gov/appeears/api/bundle/098dbb7a-0dfc-4f19-8410-a1db4f91170c/5a51d0f8-362a-4ea3-9e8e-ce1bd783aa62/MCD12Q1-006-LC-Prop1-Statistics.csv')
qrf.to_sql

In [ ]:
##close the connection
cnx.close()